# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
# PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# PATH_TO_CKPT = '/home/mahesh/davg/tensorflow_api/fine_tuned_model' + '/frozen_inference_graph.pb'
PATH_TO_CKPT = '/home/mahesh/davg/faster_rcnn_inception_v2/fine_tuned_model_yolo' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = '/home/mahesh/davg/faster_rcnn_inception_v2/data/label_map.pbtxt'

# NUM_CLASSES = 90
NUM_CLASSES = 1

## Download Model

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

NameError: name 'DOWNLOAD_BASE' is not defined

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
import cv2
import glob
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 7) ]


images_path = glob.glob('/home/mahesh/davg/images/**/*.jpg', recursive=True)

print(images_path[0][25:])

TEST_IMAGE_PATHS = images_path
im1 = cv2.imread(images_path[0])
                 

SAVE_PRED_IMAGES_PATH = '/home/mahesh/davg/images_pred/'
# cv2.imwrite(SAVE_PRED_IMAGES_PATH + images_path[0][25:], im1)
# Size, in inches, of the output images.
# IMAGE_SIZE = (12, 8)
IMAGE_SIZE = (6,4)

stopsign341.jpg


In [14]:
import datetime
import numpy as np
# time_taken_array = []
time_taken = []
time_taken_array = np.array(time_taken, dtype = np.float32)

# floatTemperatures = np.append(floatTemperatures, TEMP1)

In [15]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      print(image_path[25:])
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      before = datetime.datetime.now()
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      time_taken = datetime.datetime.now() - before
      time_taken_array = np.append(time_taken_array, time_taken.total_seconds())
#           print(boxes, scores, classes, num)
      # Visualization of the results of a detection.
    
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
#       plt.figure(figsize=IMAGE_SIZE)
#       plt.imshow(image_np)
#       print(type(image_np))
#       print(image_np.shape)
#       cv2.imwrite(SAVE_PRED_IMAGES_PATH + image_path[25:], cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
      
#       print(boxes, scores, classes, num)

stopsign341.jpg
stopsign288.jpg
rural_080.jpg
stopsign3.jpg
105.jpg
061.jpg
085.jpg
rural_026.jpg
city_074.jpg
073.jpg
rural_005.jpg
stopsign311.jpg
stopsign137.jpg
013.jpg
019.jpg
003.jpg
stopsign42.jpg
rural_002.jpg
city_015.jpg
stopsign164.jpg
009.jpg
rural_019.jpg
stopsign189.jpg
rural_048.jpg
rural_076.jpg
stopsign313.jpg
stopsign115.jpg
stopsign318.jpg
089.jpg
rural_021.jpg
098.jpg
city_036.jpg
055.jpg
stopsign348.jpg
stopsign27.jpg
097.jpg
city_038.jpg
stopsign192.jpg
stopsign118.jpg
008.jpg
stopsign37.jpg
stopsign102.jpg
stopsign182.jpg
091.jpg
stopsign351.jpg
city_019.jpg
rural_065.jpg
stopsign123.jpg
stopsign34.jpg
rural_042.jpg
city_045.jpg
stopsign282.jpg
city_076.jpg
stopsign314.jpg
rural_044.jpg
stopsign158.jpg
051.jpg
city_068.jpg
stopsign374.jpg
stopsign33.jpg
106.jpg
city_027.jpg
rural_058.jpg
029.jpg
stopsign136.jpg
stopsign340.jpg
036.jpg
stopsign327.jpg
stopsign339.jpg
rural_046.jpg
city_054.jpg
city_008.jpg
stopsign30.jpg
stopsign124.jpg
094.jpg
stopsign333.jpg
066

In [16]:
for time in time_taken_array:
    print(time)

0.377279
0.01208
0.01227
0.01328
0.021206
0.010676
0.011704
0.010559
0.011207
0.014283
0.01153
0.015474
0.011983
0.013431
0.014595
0.012026
0.011333
0.013558
0.011884
0.014168
0.014295
0.0136
0.014782
0.014678
0.018599
0.016633
0.012577
0.012784
0.011976
0.012986
0.012977
0.011643
0.013083
0.011206
0.012248
0.012636
0.016051
0.014451
0.012466
0.012074
0.017549
0.013294
0.015459
0.012941
0.013602
0.014453
0.012961
0.012362
0.012552
0.012374
0.012644
0.014276
0.016428
0.019468
0.015326
0.015092
0.015446
0.012167
0.012578
0.013941
0.011351
0.010567
0.011382
0.011087
0.012295
0.010707
0.010835
0.013729
0.016709
0.014145
0.011987
0.01263
0.014911
0.012298
0.012469
0.011724
0.013859
0.012755
0.01244
0.012497
0.012303
0.015022
0.012289
0.012396
0.012161
0.01277
0.013034
0.012197
0.013138
0.016488
0.018685
0.015278
0.012512
0.014268
0.014236
0.012754
0.01459
0.017009
0.019064
0.012588
0.011389
0.013139
0.013602
0.0111
0.012562
0.012627
0.015555
0.010975
0.012066
0.011465
0.020414
0.011622
0.01